In [1]:
!pip install -q transformers datasets torch scikit-learn


In [2]:
from google.colab import files
uploaded = files.upload()


Saving heldout_pair_data.jsonlist to heldout_pair_data.jsonlist


In [3]:
!ls -lh


total 9.7M
-rw-r--r-- 1 root root 9.7M Oct 27 16:45 heldout_pair_data.jsonlist
drwxr-xr-x 1 root root 4.0K Oct 24 13:37 sample_data


In [4]:
import json, pandas as pd

data = []
path = "heldout_pair_data.jsonlist"

with open(path, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        # תגובות משכנעות
        if "positive" in obj and "comments" in obj["positive"]:
            for c in obj["positive"]["comments"]:
                text = c.get("body", "").strip()
                if text:
                    data.append({"text": text, "label": 1})
        # תגובות לא משכנעות
        if "negative" in obj and "comments" in obj["negative"]:
            for c in obj["negative"]["comments"]:
                text = c.get("body", "").strip()
                if text:
                    data.append({"text": text, "label": 0})

df = pd.DataFrame(data)
print(df.head())
print("Total samples:", len(df))
print(df["label"].value_counts())


                                                text  label
0  In short, the reality is: It adds up.\n\n[Does...      1
1  I agree with /u/huadpe that simply the act of ...      0
2  It's not about monetary or practical impact, n...      1
3  I think I see where you are coming from, espec...      0
4  If you want cold and forests, I'd suggest Cana...      1
Total samples: 2142
label
1    1134
0    1008
Name: count, dtype: int64


In [5]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df["label"], random_state=42)

ds = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(val_df),
    "test": Dataset.from_pandas(test_df)
})


In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)

tokenized = ds.map(tokenize, batched=True)
tokenized = tokenized.rename_column("label", "labels")
tokenized.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/1713 [00:00<?, ? examples/s]

Map:   0%|          | 0/214 [00:00<?, ? examples/s]

Map:   0%|          | 0/215 [00:00<?, ? examples/s]

In [8]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy":  accuracy_score(labels, preds),
        "f1":        f1_score(labels, preds),
        "precision": precision_score(labels, preds),
        "recall":    recall_score(labels, preds),
    }


In [10]:
!pip install -U transformers==4.57.1
import importlib, transformers
importlib.reload(transformers)
print(transformers.__version__)


4.57.1


In [15]:
from transformers import Trainer, TrainingArguments
import os

args = TrainingArguments(
    output_dir="./results",
    logging_dir="./logs",
    logging_steps=100,
    eval_steps=200,                 # הערכה כל 200 צעדים
    save_steps=200,                 # שמירה כל 200 צעדים
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    load_best_model_at_end=False,   # אין מעקב אחרי המודל הכי טוב בגרסה הזו
)


os.environ["WANDB_DISABLED"] = "true"

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()
print("Validation:", trainer.evaluate(tokenized["validation"]))
print("Test:", trainer.evaluate(tokenized["test"]))




Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-2218503766.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
100,0.705700
200,0.691900
300,0.664000
400,0.688000
500,0.636000
600,0.621700


Validation: {'eval_loss': 0.7038731575012207, 'eval_accuracy': 0.5654205607476636, 'eval_f1': 0.5633802816901409, 'eval_precision': 0.6, 'eval_recall': 0.5309734513274337, 'eval_runtime': 3.0315, 'eval_samples_per_second': 70.592, 'eval_steps_per_second': 8.906, 'epoch': 3.0}
Test: {'eval_loss': 0.6730859875679016, 'eval_accuracy': 0.586046511627907, 'eval_f1': 0.6147186147186147, 'eval_precision': 0.6068376068376068, 'eval_recall': 0.6228070175438597, 'eval_runtime': 3.01, 'eval_samples_per_second': 71.428, 'eval_steps_per_second': 8.97, 'epoch': 3.0}


In [16]:
print("Validation:", trainer.evaluate(tokenized["validation"]))
print("Test:", trainer.evaluate(tokenized["test"]))


Validation: {'eval_loss': 0.7038731575012207, 'eval_accuracy': 0.5654205607476636, 'eval_f1': 0.5633802816901409, 'eval_precision': 0.6, 'eval_recall': 0.5309734513274337, 'eval_runtime': 3.1919, 'eval_samples_per_second': 67.045, 'eval_steps_per_second': 8.459, 'epoch': 3.0}
Test: {'eval_loss': 0.6730859875679016, 'eval_accuracy': 0.586046511627907, 'eval_f1': 0.6147186147186147, 'eval_precision': 0.6068376068376068, 'eval_recall': 0.6228070175438597, 'eval_runtime': 3.1978, 'eval_samples_per_second': 67.233, 'eval_steps_per_second': 8.443, 'epoch': 3.0}
